In [1]:
import pandas as pd
import json

In [2]:
with open('./model_abtest.log','r') as f:
    raw = f.read().split('\n')[:-1]

In [3]:
data_list = []
for event in raw:
    obs = json.loads(event)
    obs['extra_data'] = obs.get('extra_data',None)
    data_list.append(obs)

In [4]:
df = pd.DataFrame.from_dict(data_list) 
df.head()

,name,time,salt,inputs,params,event,checksum,extra_data
0,ModelExperiment,1602739669,ModelExperiment,{'userid': '431'},"{'use_pytorch': 1, 'model_type': 'pytorch'}",exposure,796b9a12,None
1,ModelExperiment,1602739720,ModelExperiment,{'userid': '431'},"{'use_pytorch': 1, 'model_type': 'pytorch'}",exposure,796b9a12,None
2,ModelExperiment,1602739722,ModelExperiment,{'userid': '431'},"{'use_pytorch': 1, 'model_type': 'pytorch'}",exposure,796b9a12,None
3,ModelExperiment,1602739722,ModelExperiment,{'userid': '431'},"{'use_pytorch': 1, 'model_type': 'pytorch'}",exposure,796b9a12,None
4,ModelExperiment,1602739724,ModelExperiment,{'userid': '637'},"{'use_pytorch': 0, 'model_type': 'surprise'}",exposure,796b9a12,None


In [5]:
exp_df = df.loc[df.event=='rate',['params','extra_data']]
exp_df['variant'] = exp_df['params'].apply(lambda x: x['use_pytorch'])
exp_df['rating'] = exp_df['extra_data'].apply(lambda x: x['rate_val'])

In [6]:
exp_df.head()

,params,extra_data,variant,rating
32,"{'use_pytorch': 0, 'model_type': 'surprise'}",{'rate_val': 3},0,3
36,"{'use_pytorch': 1, 'model_type': 'pytorch'}",{'rate_val': 7},1,7
41,"{'use_pytorch': 1, 'model_type': 'pytorch'}",{'rate_val': 0},1,0
44,"{'use_pytorch': 0, 'model_type': 'surprise'}",{'rate_val': 7},0,7
48,"{'use_pytorch': 0, 'model_type': 'surprise'}",{'rate_val': 3},0,3


In [7]:
d1 = exp_df.loc[exp_df.variant==0,'rating'].values
d2 = exp_df.loc[exp_df.variant==1,'rating'].values

In [8]:
import scipy.stats
statistic, pval = scipy.stats.ttest_ind(d1,d2)
print(statistic,pval)

0.9650813700331365 0.37882815062317066


In [9]:
import random
d1 = random.choices(range(1,6),k=50)
d2 = random.choices(range(3,10),k=50)

In [10]:
scipy.stats.ttest_ind(d1,d2)

Ttest_indResult(statistic=-10.876400571212649, pvalue=1.5325813144977069e-18)